# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
# import libraries
import sqlalchemy as sql
import pandas as pd
import numpy as np
import re
import pickle 

In [14]:
# load data from database
engine =sql.create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',con=engine)
#df
X = df.message
Y = df.iloc[:,4:]
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [37]:
import nltk
nltk.download(['punkt','wordnet', 'averaged_perceptron_tagger'])
def tokenize(text):
    url='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    found_url = re.findall(url,text)
    for i in found_url:
        text=text.replace(i,"<URL>")
    tags = nltk.tokenize.word_tokenize(text)
    lemmatized = nltk.stem.WordNetLemmatizer()
    
    final_tags = []
    for i in tags:
        final_tag = lemmatized.lemmatize(i).lower().strip()
        final_tags.append(final_tag)
    return final_tags

[nltk_data] Downloading package punkt to /home/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/aditya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [40]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

#Test model Pipeline
def Pipeline_test():
    pipeline = Pipeline([
        ('vectorize',CountVectorizer(tokenizer=tokenize)),
        ('Tfidf',TfidfTransformer()),
        ('classifier', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline
         

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = Pipeline_test()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

#### As we can see the test model is doing decent job but if we could make it more better!

In [53]:
#sample Testing on the model 

msg1 = ['Please I nead help, I am stuck in the house and it has caught fire']
msg2 = ['Hello I see fire in the street and many houses are destroyed, lots of fire I people everywhere']
output1 = model.predict(msg1)
output2=model.predict(msg2)
#y_train
print(y_train.columns.values[(output1.flatten()==1)])
print(y_train.columns.values[(output2.flatten()==1)])

['related' 'request' 'aid_related']
['related' 'aid_related' 'direct_report']


#### Improvised Model

In [55]:
# Model Improvised

# Here I will add a custom estimator of extracting beginning verbs for better results
from sklearn.base import BaseEstimator, TransformerMixin
class VerbExtract(BaseEstimator, TransformerMixin):

    #Helper fucntion 
    def verb_begin(self, text):
        sentence = nltk.sent_tokenize(text)
        for s in sentence:
            pos_tags = nltk.pos_tag(tokenize(s))
            f_word, f_tag = pos_tags[0]
            if f_tag in ['VB', 'VBP'] or f_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tag = pd.Series(X).apply(self.verb_begin)
        return pd.DataFrame(X_tag)

In [56]:
#Here I introduce new classifier AdaBoost
from sklearn.ensemble import AdaBoostClassifier
def model_imporvised():
    pipeline = Pipeline([
        ('Features', FeatureUnion([

            ('textpipeline', Pipeline([
                ('vectorize', CountVectorizer(tokenizer=tokenize)),
                ('Ffidf', TfidfTransformer())
            ])),

            ('begin_verb', VerbExtract())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [57]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = []
model = Pipeline_test()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                    

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [60]:
y_pred=model.predict(X_test)

In [61]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats.mstats import gmean

def fscore_method(y_true,y_pred,b=1):
    scores = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta=b,average='weighted')
        scores.append(score)
    f1score_numpy = np.asarray(scores)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

In [63]:
f1 = fscore_method(y_test,y_pred, b= 1)
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(f1*100))

Average accuracy 94.62% 

F1 score 92.81%



In [64]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('**************************************************\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

**************************************************

FEATURE: related

              precision    recall  f1-score   support

           0       0.72      0.26      0.38      1571
           1       0.80      0.97      0.87      4935
           2       0.67      0.08      0.15        48

    accuracy                           0.79      6554
   macro avg       0.73      0.44      0.47      6554
weighted avg       0.78      0.79      0.75      6554

**************************************************

FEATURE: request

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5421
           1       0.89      0.43      0.58      1133

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

**************************************************

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      

/home/aditya/miniconda3/envs/dl/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6344
           1       0.62      0.05      0.09       210

    accuracy                           0.97      6554
   macro avg       0.80      0.52      0.54      6554
weighted avg       0.96      0.97      0.96      6554

**************************************************

FEATURE: child_alone

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

**************************************************

FEATURE: water

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6129
           1       0.91      0.22      0.36       425

    accuracy                           0.95      6554
   macro avg       0.93      0.61      0

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6484
           1       1.00      0.01      0.03        70

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

**************************************************

FEATURE: earthquake

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5964
           1       0.90      0.74      0.81       590

    accuracy                           0.97      6554
   macro avg       0.93      0.86      0.90      6554
weighted avg       0.97      0.97      0.97      6554

**************************************************

FEATURE: cold

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6432
           1       0.80      0.07      0.12       122

    accuracy                           0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [82]:
from sklearn.model_selection import GridSearchCV
model = model_imporvised()

parameters = {
    'Features__textpipeline__vectorize__ngram_range': ((1, 1), (1, 2)),
    'Features__textpipeline__vectorize__max_df': (0.75, 1.0),
    'Features__textpipeline__vectorize__max_features': (None, 5000),
    'Features__textpipeline__Ffidf__use_idf': (True, False),
    
  }

scorer=make_scorer(fscore_method,greater_is_better = True)

cv = GridSearchCV(model, param_grid=parameters, scoring = scorer,verbose = 2, n_jobs = -1)
cv.fit(X_train,y_train)
#cv.get_params().keys()

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 50.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('Features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('textpipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vectorize',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
           

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

#### As we can see now the model predicts with same messages as old ones far better!!!

In [84]:
msg1 = ['Please I nead help, I am stuck in the house and it has caught fire']
msg2 = ['Hello I see fire in the street and many houses are destroyed, lots of fire I people everywhere']
output1 = cv.predict(msg1)
output2=cv.predict(msg2)
#y_train
print(y_train.columns.values[(output1.flatten()==1)])
print(y_train.columns.values[(output2.flatten()==1)])

['related' 'request' 'aid_related' 'other_aid' 'weather_related'
 'direct_report']
['related' 'aid_related' 'shelter' 'buildings' 'weather_related'
 'direct_report']


### I can see Some good amount of accuracy increase as well as F1 Score after grid searchCV

In [85]:
y_pred=cv.predict(X_test)
f1 = fscore_method(y_test,y_pred, b= 1)
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(f1*100))

Average accuracy 94.78% 

F1 score 93.72%



### 9. Export your model as a pickle file

In [86]:
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.